# Creación y Almacenamiento de Embeddings en ChromaDB

En el presente Notebook se crearán la documentación base del esquema reducido con el que trabajaremos, basada en el `information_schema` correspondiente, que luego se nutrirá por fuera con mayor información relevante para negocio. 

Adicionalmente, por fuera del Notebook, se confeccionará documentación adicional, como reglas de negocio y few-shot examples para, finalmente, obtener chunks de todos estos documentos y crear y almacenar los mismos, con su metadata asociada, en una base de datos vectorial de ChromaDB.

---

## Inicialización

### Librerías



In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import regex
import yaml
import json
from IPython.display import Markdown

from copy import deepcopy
from enum import Enum
from typing import Any
from collections import defaultdict, OrderedDict
import pandas as pd

import uuid
import chromadb
from google import genai

from langchain_core.documents import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import Language, MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings.base import Embeddings as LangchainEmbeddings
from langchain_chroma import Chroma

from langchain.prompts import ChatPromptTemplate
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.schema import StrOutputParser


load_dotenv('../.env')
pd.set_option('display.max_columns', None)
yaml.add_representer(OrderedDict, lambda dumper, data: dumper.represent_dict(data.items()))

notebook_dir = os.getcwd() 
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)


from src.pg_sql import execute_query


### Constantes

In [2]:
DATABASE = 'database'
SCHEMAS = 'schemas'
TABLES = 'tables'
COLUMNS = 'columns'
DESCRIPTION = 'description'
NAME = 'name'
DATA_TYPE = 'data_type'
PRIMARY_KEY = 'is_primary_key'
FOREIGN_KEY = 'is_foreign_key'
REFERENCE = 'reference'
TO_DO = '[To be completed ...]'

OUTPUT_PATH = '../data/embeddings/auxs'
MDL_PATH = '../data/embeddings/documents/MDL_adventure_works_dw.yaml'
BUSINESS_RULES_PATH = '../data/embeddings/documents/business_rules.md'
EXAMPLES_PATH = '../data/embeddings/documents/query_examples.json'

BUSINESS_RULES_CHUNK_SIZE = 750
BUSINESS_RULES_CHUNK_OVERLAP = int(0.2 * BUSINESS_RULES_CHUNK_SIZE)
HEADERS_CHUNK_ID_KEY = 'headers_chunk_id'
HEADERS_CHUNKS_TOTAL_KEY = 'headers_comb_total'
CHUNK_OVERLAP_KEY = 'chunk_overlap'
EXTRA_CHUNKS_METADATA = (HEADERS_CHUNK_ID_KEY, HEADERS_CHUNKS_TOTAL_KEY, CHUNK_OVERLAP_KEY)
MIN_KEY = 'min'
MAX_KEY = 'max'
TOTAL_KEY = 'total'

CHROMA_DB_PATH = '../data/embeddings/chroma'
TABLES_SUMMARY_COLLECTION_NAME = 'mdl_tables_summary'
COLUMNS_COLLECTION_NAME = 'mdl_columns'
BUSINESS_LOGIC_COLLECTION_NAME = 'business_logic'

AZURE_OPENAI_EMBEDDING_MODEL = 'text-embedding-3-large'
GENAI_EMBEDDING_MODEL = 'gemini-embedding-001'

### Clase personalizadas

In [3]:
class GenaiEmbedConfigTaskType(str, Enum):
    """
    Enum representing the different task types for GenAI embeddings.
    The value of each member is the same as its name.
    """
    CLASSIFICATION = 'CLASSIFICATION'
    CLUSTERING = 'CLUSTERING'
    RETRIEVAL_DOCUMENT = 'RETRIEVAL_DOCUMENT'
    RETRIEVAL_QUERY = 'RETRIEVAL_QUERY'
    QUESTION_ANSWERING = 'QUESTION_ANSWERING'
    FACT_VERIFICATION = 'FACT_VERIFICATION'
    CODE_RETRIEVAL_QUERY = 'CODE_RETRIEVAL_QUERY'
    SEMANTIC_SIMILARITY = 'SEMANTIC_SIMILARITY'



class GenAIExtendedEmbeddingFunction(LangchainEmbeddings):
    def __init__(self, model: str):
        """
        Initializes the class with the Gemini model.
        
        Args:
            model (str): The name of the Gemini model.
        """
        self.model = model
        self.genai_client = genai.Client()
    
    def _get_genai_embeddings(self, contents: list[str], task_type: GenaiEmbedConfigTaskType | None = None) -> list[list[float]]:
        """
        Internal method to get embeddings from the GenAI API.
        """
        embed_config = genai.types.EmbedContentConfig(task_type=task_type.value) if task_type else None
        
        response = self.genai_client.models.embed_content(
            model= self.model,
            contents= contents,
            config= embed_config
        )
        
        return [embed.values for embed in response.embeddings]
        
    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        """
        Generates embeddings for a list of documents.
        """
        return self._get_genai_embeddings(
            contents= texts,
            task_type= GenaiEmbedConfigTaskType.RETRIEVAL_DOCUMENT
        )

    def embed_query(self, text: str) -> list[float]:
        """
        Generates an embedding for a single query string.
        """
        return self._get_genai_embeddings(
            contents= [text],
            task_type= GenaiEmbedConfigTaskType.RETRIEVAL_QUERY
        )[0]


class DocumentContextEnricher:
    """A class to retrieve, merge, and enrich document contexts from a ChromaDB collection.

    This class encapsulates the logic for expanding a semantic search result set
    by retrieving a larger context window of chunks for each unique document.
    It handles document grouping, context window calculation, and text
    reconstruction by intelligently merging overlapping chunks.
    """

    def __init__(self, chroma_collection: Chroma) -> None:
        """
        Initializes the DocumentContextEnricher with a ChromaDB collection.

        Args:
            chroma_collection (Chroma): The ChromaDB collection instance to query.
        """
        self.chroma_collection = chroma_collection


    def _get_metadata_key_value_sorted(self, metadata: dict, 
                                       exclude_keys: tuple|list = None) -> tuple[tuple[str, Any]]:
        """Sorts and filters a metadata dictionary for consistent comparison.

        This utility function creates a canonical representation of a metadata
        dictionary by sorting its key-value pairs. It's intended for use as a
        dictionary key to group similar documents.

        Args:
            metadata (dict): The dictionary of metadata to process.
            exclude_keys (tuple | list): A collection of keys to exclude
                                         from the final sorted output.

        Returns:
            tuple[tuple[str, Any]]: A sorted tuple of (key, value) pairs,
                                    excluding the specified keys.
        """

        if not exclude_keys:
            exclude_keys = []
        
        return tuple(sorted((k, v) for k, v in metadata.items() if k not in exclude_keys))


    def _get_chunk_limits(self, md_min: int, md_max: int, 
                          md_total: int, k_window: int) -> tuple[int, int]:
        """Determines the start and end chunk indices for a given context window.

        This function calculates the valid start (inclusive) and end (inclusive)
        indices for a context window of size `k_window` based on a document's
        position (`md_min` and `md_max`) and the total number of documents.
        It handles edge cases for the beginning and end of the document set.

        Args:
            md_min (int): The starting index of the document.
            md_max (int): The ending index of the document.
            md_total (int): The total number of documents (0-indexed).
            k_window (int): The desired size of the context window.

        Returns:
            tuple[int, int]: A tuple containing the start and end chunk indices
                             (inclusive) for the context window.
        """
        chunk_count = md_max - md_min + 1
        if chunk_count >= k_window:
            return (md_min, md_max)

        remaining_needed = k_window - chunk_count
        add_before = remaining_needed // 2
        add_after = remaining_needed - add_before
        
        start_limit = md_min - add_before
        end_limit = md_max + add_after
        
        if start_limit < 0:
            end_limit += abs(start_limit)
            start_limit = 0
        
        if end_limit > md_total:
            start_limit -= (end_limit - md_total)
            end_limit = md_total
            if start_limit < 0:
                start_limit = 0
                
        return (start_limit, end_limit)
    

    def _merge_overlap_chunks(self, docs: dict) -> str:
        """Merges a list of overlapping document chunks into a single string.

        This function takes a dictionary of document metadata and content,
        sorts the chunks by their `headers_chunk_id`, and then concatenates
        them while removing the overlapping parts by dynamically finding
        the overlap size.

        Args:
            docs (dict): A dictionary containing 'metadatas' and 'documents'
                from a ChromaDB query.

        Returns:
            str: The final, merged string of all document chunks with overlaps
                removed.
        """

        if not docs.get('documents'):
            return ''

        chunks_data = []
        for metadata, content in zip(docs['metadatas'], docs['documents']):
            chunks_data.append(Document(metadata=metadata, page_content=content))

        sorted_chunks = sorted(chunks_data, key=lambda doc: doc.metadata.get(HEADERS_CHUNK_ID_KEY, 0))
        
        def find_overlap(chunk1: str, chunk2: str, max_overlap: int) -> int:
            """
            Finds the length of the longest overlap between the end of chunk1 and the start of chunk2.

            Args:
                chunk1: The first string.
                chunk2: The second string.
                max_overlap: The maximum allowed overlap length.

            Returns:
                The length of the longest overlap.
            """
            for i in range(min(len(chunk1), max_overlap), 0, -1):
                if chunk1.endswith(chunk2[:i]):
                    return i
            return 0

        reconstructed_parts = [sorted_chunks[0].page_content]
        max_overlap_size = sorted_chunks[0].metadata.get(CHUNK_OVERLAP_KEY, 0)
        
        for i in range(1, len(sorted_chunks)):
            prev_chunk_content = sorted_chunks[i-1].page_content
            current_chunk_content = sorted_chunks[i].page_content
            
            overlap_size = find_overlap(prev_chunk_content, current_chunk_content, max_overlap_size)
            
            reconstructed_parts.append(current_chunk_content[overlap_size:])
            
        return "".join(reconstructed_parts)


    def get_enriched_context_chunks(
        self,
        semantic_results: list[tuple[Document, float] | Document],
        k_chunks_context_window: int = 3
    ) -> list[str]:
        """Retrieves and merges chunks for a larger context window based on semantic search results.

        This function takes a list of semantic search results, identifies unique
        document groups based on their metadata, and for each group, retrieves
        a larger context window of chunks from a Chroma collection. It then
        merges the overlapping chunks to form a cohesive, larger document context.

        Args:
            semantic_results (list[tuple[Document, float] | Document]): 
                A list of tuples containing a Document object and its 
                similarity score from a semantic search.
            k_chunks_context_window (int): 
                The size of the context window to retrieve around each result.

        Returns:
            list[str]: A list of merged strings, where each string represents a
                       full document context from a unique source document.
        """
        if not semantic_results:
            return []

        processed_results_dict = defaultdict(lambda: {
            'min_id': float('inf'),
            'max_id': float('-inf'),
            'total_chunks': None
        })

        docs = (
            deepcopy(semantic_results)
            if isinstance(semantic_results[0], Document)
            else [doc for doc, _ in semantic_results]
        )

        for doc in docs:
            metadata_key = self._get_metadata_key_value_sorted(doc.metadata, EXTRA_CHUNKS_METADATA)
            chunk_id = doc.metadata.get(HEADERS_CHUNK_ID_KEY)
            total_chunks = doc.metadata.get(HEADERS_CHUNKS_TOTAL_KEY)

            if chunk_id is not None and total_chunks is not None:
                processed_results_dict[metadata_key]['min_id'] = min(processed_results_dict[metadata_key]['min_id'], chunk_id)
                processed_results_dict[metadata_key]['max_id'] = max(processed_results_dict[metadata_key]['max_id'], chunk_id)
                processed_results_dict[metadata_key]['total_chunks'] = total_chunks

        output_chunks = []
        for metadata_tuple, chunk_data in processed_results_dict.items():
            min_limit, max_limit = self._get_chunk_limits(
                chunk_data['min_id'],
                chunk_data['max_id'],
                chunk_data['total_chunks'],
                k_window=k_chunks_context_window
            )

            conditions = [{k: {'$eq': v}} for k, v in metadata_tuple]
            conditions.append({HEADERS_CHUNK_ID_KEY: {"$gte": min_limit}})
            conditions.append({HEADERS_CHUNK_ID_KEY: {"$lte": max_limit}})

            window_results = self.chroma_collection.get(
                where={"$and": conditions}
            )

            output_chunks.append(self._merge_overlap_chunks(window_results))

        return output_chunks
    


### Funciones aplicables a todos los documentos

In [4]:
def add_docs_to_chroma_col(chunks: list[dict], collection: Chroma) -> list[str]:
    """
    Store a list of document chunks into a ChromaDB collection using langchain.

    Each chunk is expected to be a dictionary containing 'content' (text of the chunk)
    and 'metadata' (associated metadata) keys. The function generates unique IDs and, if required,
    computes embeddings for each document chunk before adding them to the collection.

    Args:
        chunks (list[dict]): A list of dictionaries, each with keys:
                             - 'content' (str): The text content of the chunk.
                             - 'metadata' (dict): Metadata associated with the chunk.
        collection (langchain_chroma.Chroma): The target ChromaDB collection by langchain to store the data in.

    Returns:
        ids[list[str]]
    """

    ids = [str(uuid.uuid4()) for _ in chunks]

    collection.add_documents(
        ids = ids,
        documents= chunks
    )



def cosine_distance_relevance_score_fn(distance: float) -> float:
    """Normalize the distance [0, 2] to a score on a scale [0, 1]."""
    return 1.0 - distance / 2


---

## Fichero YAML con MDL del esquema de interés

### Obtener fichero base desde `information_schema`

Primero obtendremos un fichero base construido utilizando la query `/data/embeddings/auxs/get_information_schema.sql`, sobre el que luego se añadirá metadata extra. Para esto, definimos algunas funciones que nos serán de utilidad:

In [4]:
def get_information_schema(query_path: str, db_names_list: list[str], schema_names_list: list[str]) -> str:
    """
    Reads an SQL query from a file and replaces placeholder lists with formatted strings.

    This function is designed to work with SQL queries that have specific placeholders
    for database and schema names. It reads the query from the given file path, 
    formats the input lists of names into a single quoted, comma-separated string, 
    and replaces the placeholders in the query.

    Args:
        query_path (str): The file path to the SQL query. The query should
                          contain the placeholders `[db_names_list]` and
                          `[schema_names_list]`.
        db_names_list (list[str]): A list of database names to be formatted
                                   and inserted into the query.
        schema_names_list (list[str]): A list of schema names to be formatted
                                       and inserted into the query.

    Returns:
        str: The complete SQL query with the placeholders replaced by
             the formatted database and schema names.

    Raises:
        FileNotFoundError: If the specified query_path does not exist.
        
    Example:
        >>> from pathlib import Path
        >>> # Assume 'my_query.sql' contains:
        >>> # SELECT * FROM information_schema.tables WHERE table_schema IN ([schema_names_list])
        >>> # And we create a dummy file for the example:
        >>> Path('my_query.sql').write_text("SELECT * FROM information_schema.tables WHERE table_schema IN ([schema_names_list])")
        >>> db_list = ['db1', 'db2']
        >>> schema_list = ['schema_a', 'schema_b']
        >>> get_information_schema('my_query.sql', db_list, schema_list)
        "SELECT * FROM information_schema.tables WHERE table_schema IN ('schema_a', 'schema_b')"
    """
    query = Path(query_path).read_text()

    db_names = "'" + "', '".join(db_names_list) + "'"
    schema_names = "'" + "', '".join(schema_names_list) + "'"

    return query.replace('[db_names_list]', db_names).replace('[schema_names_list]', schema_names)



def format_yaml(yaml_str: str) -> str:
    """
    Formats a YAML string by adding a blank line before each list item
    that isn't preceded by a list key.
    """
    last_line = ''
    last_line_list_init = False
    last_line_empty = False

    lines = list()

    for line in yaml_str.split('\n'):
        if line.strip().startswith('-') and not last_line_list_init and not last_line_empty:
            last_line += '\n'

        lines.append(last_line)
        last_line = line
        last_line_list_init = last_line.strip().endswith(':')
        last_line_empty = last_line.strip()==''

    lines.append(line)

    return '\n'.join(lines)

In [5]:
GET_INFORMATION_SCHEMA_SQL = '../data/embeddings/auxs/get_information_schema.sql'
DB_NAME = 'adventure_works_dw'
SCHEMA_NAME = 'sales'


information_schema_data = execute_query(get_information_schema(
    query_path= GET_INFORMATION_SCHEMA_SQL,
    db_names_list= [DB_NAME],
    schema_names_list= [SCHEMA_NAME]
))

Veamos el aspecto que tienen los resultados de nuestra query:

In [6]:
information_schema_data

[{'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'customer_key',
  'column_type': 'INT4',
  'primary_key': True,
  'foreign_key': False,
  'target': None},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'geography_key',
  'column_type': 'INT4',
  'primary_key': False,
  'foreign_key': True,
  'target': 'sales.dim_geography.geography_key'},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'customer_full_name',
  'column_type': 'TEXT',
  'primary_key': False,
  'foreign_key': False,
  'target': None},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
  'column_name': 'birth_date',
  'column_type': 'DATE',
  'primary_key': False,
  'foreign_key': False,
  'target': None},
 {'db_name': 'adventure_works_dw',
  'schema_name': 'sales',
  'table_name': 'dim_customer',
 

Lo convertimos en un Data Frame de Pandas para que sea más vistoso:

In [7]:
pd.DataFrame(information_schema_data)

,db_name,schema_name,table_name,column_name,column_type,primary_key,foreign_key,target
0,adventure_works_dw,sales,dim_customer,customer_key,INT4,True,False,None
1,adventure_works_dw,sales,dim_customer,geography_key,INT4,False,True,sales.dim_geography.geography_key
2,adventure_works_dw,sales,dim_customer,customer_full_name,TEXT,False,False,None
3,adventure_works_dw,sales,dim_customer,birth_date,DATE,False,False,None
4,adventure_works_dw,sales,dim_customer,marital_status,BPCHAR(1),False,False,None
...,...,...,...,...,...,...,...,...
107,adventure_works_dw,sales,fact_sales,freight,NUMERIC,False,False,None
108,adventure_works_dw,sales,fact_sales,order_date,DATE,False,False,None
109,adventure_works_dw,sales,fact_sales,due_date,DATE,False,False,None
110,adventure_works_dw,sales,fact_sales,ship_date,DATE,False,False,None


Ahora procederemos a crear el YAML base con el MDL de nuestro esquema, que podremos tomar como punto de partida para luego añadirle metadata extra manualmente:

In [ ]:
dbs_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for row in information_schema_data:
    db_name = row.get('db_name')
    schema_name = row.get('schema_name')
    table_name = row.get('table_name')
    
    dbs_data[db_name][schema_name][table_name].append(row)

for db_name, schemas_data in dbs_data.items():
    db = OrderedDict()
    db[DATABASE] = db_name
    db[DESCRIPTION] = TO_DO

    schemas = list()
    for schema_name, tables_data in schemas_data.items():
        schema = OrderedDict()
        schema[NAME] = schema_name
        schema[DESCRIPTION] = TO_DO

        tables = list()
        for table_name, columns_data in tables_data.items():
            table = OrderedDict()
            table[NAME] = table_name
            table[DESCRIPTION] = TO_DO

            columns = list()
            for column_data in columns_data:
                column = OrderedDict()
                column[NAME] = column_data.get('column_name')
                column[DESCRIPTION] = TO_DO
                column[DATA_TYPE] = column_data.get('column_type')
                
                if column_data.get('primary_key'):
                    column[PRIMARY_KEY] = True

                if column_data.get('foreign_key'):
                    column[FOREIGN_KEY] = True
                    column[REFERENCE] = column_data.get('target')

                columns.append(column)

            table[COLUMNS] = columns
            tables.append(table)
        
        schema[TABLES] = tables
        schemas.append(schema)
    
    db[SCHEMAS] = schemas

    mdl_file_path = f'{OUTPUT_PATH}/MDL_{db_name}.yaml'
    with open(mdl_file_path, 'w') as mdl:
        mdl.write(format_yaml(yaml.dump(db)))

    print(f'>  Fichero MDL base almacenado en {mdl_file_path}')

>  Fichero MDL base almacenado en ../data/embeddings/auxs/MDL_adventure_works_dw.yaml


### Creación de embeddings

#### Chunking

Tomando como base el fichero obtenido en el apartado anterior, se ha creado un fichero que del MDL de nuestro esquema de interés que ha sido nutrido con metadata adicional, como descripciones para cada tabla y campo. Este fichero se encuentra en la ruta `/data/embeddings/documents/MDL_adventure_works_dw.yaml`.

Procederemos ahora a procesar este fichero para crear chunks de la información de cada una de las tablas y almacenar sus embeddings en nuestra base de datos vectorial de `chroma_db`, persistida en la ruta `/data/embeddings/chroma_db/`.

Procedemos ahora a definir funciones que nos permitirán chunkear el documento mdl desde un enfoque más integral a nivel de tablas, como también a un enfoque más granular a nivel de columnas:

In [5]:
def chunk_mdl_by_table_summary(mdl_data: dict) -> list[Document]:
    """
    Chunks a Model-Driven Library (MDL) data structure into a list of Documents,
    with each document representing a summary of a single table.

    Each Document's page_content contains a summary of the table, including its
    database, schema, table name, description, primary keys, and foreign keys.
    The Document's metadata includes the database, schema, and table name for
    easy lookup and filtering.

    Args:
        mdl_data: A dictionary representing the MDL data structure. It is expected
                  to contain keys for 'database', 'schemas', 'tables', 'columns',
                  and other relevant metadata.

    Returns:
        A list of Document objects, where each object holds a table summary chunk
        from the input MDL data.
    """

    chunks = []

    database_name = mdl_data[DATABASE]

    for schema in mdl_data[SCHEMAS]:
        schema_name = schema[NAME]

        for table in schema[TABLES]:
            table_name = table[NAME]
            table_description = table[DESCRIPTION]
            table_columns = table[COLUMNS]
            
            chunk_content_list = [
                f'Database: {database_name}',
                f'Schema: {schema_name}',
                f'Table: {table_name}',
                f'Table description: {table_description}'
            ]

            table_primary_key = [col[NAME] for col in table_columns if col.get(PRIMARY_KEY)]
            if table_primary_key:
                chunk_content_list.append('Table PRIMARY KEY:')
                for pk in table_primary_key:
                    chunk_content_list.append(f'- {pk}')

            table_foreign_keys = [{'column_name': col[NAME], 'reference': col[REFERENCE]} for col in table_columns if col.get(FOREIGN_KEY)]
            if table_foreign_keys:
                chunk_content_list.append('Table FOREIGN KEYS (Column name, Reference):')
                for fk in table_foreign_keys:
                    chunk_content_list.append(f'- ({fk["column_name"]}, {fk["reference"]})')

            chunk_content = '\n'.join(chunk_content_list)

            metadata = {
                'database_name': database_name,
                'schema_name': schema_name,
                'table_name': table_name
            }

            chunks.append(Document(page_content= chunk_content, metadata= metadata))

    return chunks



def chunk_mdl_by_column(mdl_data: dict) -> list[Document]:
    """
    Chunks a Model-Driven Library (MDL) data structure into a list of Documents,
    with each document representing a single column.

    This function iterates through the schemas, tables, and columns of the MDL
    data. For each non-key column (excluding primary and foreign keys), it
    creates a Document. The Document's page_content contains the column's
    name, data type, and description. The Document's metadata includes
    the database, schema, table, and column details for easy retrieval and
    filtering.

    Args:
        mdl_data: A dictionary representing the MDL data structure. It is expected
                  to contain keys for 'database', 'schemas', 'tables', 'columns',
                  and other relevant metadata.

    Returns:
        A list of Document objects, where each object holds a chunk of data
        for a non-key column from the input MDL data.
    """
    chunks = []

    database_name = mdl_data[DATABASE]

    for schema in mdl_data[SCHEMAS]:
        schema_name = schema[NAME]

        for table in schema[TABLES]:
            table_name = table[NAME]
            table_columns = table[COLUMNS]

            for col in table_columns:
                chunk_content_list = list()

                if col.get(PRIMARY_KEY) or col.get(FOREIGN_KEY):
                    continue

                column_name = col[NAME]
                column_data_type = col[DATA_TYPE]
                column_description = col[DESCRIPTION]

                chunk_content_list.append(f'Column name: {column_name}')
                chunk_content_list.append(f'Column data type: {column_data_type}')
                chunk_content_list.append(f'Column description: {column_description}')

                chunk_content = '\n'.join(chunk_content_list)

                metadata = {
                    'database_name': database_name,
                    'schema_name': schema_name,
                    'table_name': table_name,
                    'column_name': column_name,
                    'column_data_type': column_data_type
                }

                chunks.append(Document(page_content= chunk_content, metadata= metadata))

    return chunks


Veamos ahora el aspecto que tiene, por ejemplo, el chunk de mayor tamaño para cada uno de los tipos:

In [6]:
with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)

output_chunks = chunk_mdl_by_table_summary(mdl_data)

max_len = max([len(c.page_content) for c in output_chunks])
print(f'{max_len=}')
print()
print([c.page_content for c in output_chunks if len(c.page_content) == max_len][0])
print()
print([c.metadata for c in output_chunks if len(c.page_content) == max_len][0])

max_len=1020

Database: adventure_works_dw
Schema: sales
Table: fact_sales
Table description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.
Table PRIMARY KEY:
- sales_order_number
- sales_order_line_number
Table FOREIGN KEYS (Column name, Reference):
- (product_key, sales.dim_product.product_key)
- (reseller_key, sales.dim_reseller.reseller_key)
- (employee_key, sales.dim_sales_person.employee_key)
- (customer_key, sales.dim_customer.customer_key)
- (promotion_key, sales.dim_promotion.promotion_key)
- (sales_territory_key, sales.dim_sa

In [7]:
with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)

output_chunks = chunk_mdl_by_column(mdl_data)

max_len = max([len(c.page_content) for c in output_chunks])
print(f'{max_len=}')
print()
print([c.page_content for c in output_chunks if len(c.page_content) == max_len][0])
print()
print([c.metadata for c in output_chunks if len(c.page_content) == max_len][0])

max_len=265

Column name: sale_source
Column data type: TEXT
Column description: Indicador de la fuente por la que ha sido realizado el pedido. reseller_sales=Pedido realizado por un vendedor para una tienda/distribuidor, internet_sales=Pedido realizado en línea por un cliente.

{'database_name': 'adventure_works_dw', 'schema_name': 'sales', 'table_name': 'fact_sales', 'column_name': 'sale_source', 'column_data_type': 'TEXT'}


#### LangChain: ChromaDB + Azure OpenAI Embeddings

Ahora haremos una primera implementación de prueba de Chroma valiéndonos de funcionalidades de la librería LangChain, usando un modelo de embeddings de OpenAI a través de Azure AI Foundry.

El primer paso es crear un cliente persistente de ChromaDB, e inicializar las colecciones tanto a nivel de tablas como de columnas. Al inicializar las colecciones, especificaremos que queremos usar como métrica de distancia la distancia de Coseno, que se calcula de la siguiente manera:

$$Distancia = 1 - \frac{A \cdot B}{\|A\| \|B\|}$$

Donde:
* $A \cdot B$ es el producto punto (producto escalar) de los vectores $A$ y $B$.
* $∥A∥$ y $∥B∥$ son las magnitudes (o normas euclidianas) de los vectores $A$ y $B$, respectivamente.
​

La métrica **Distancia del Coseno** mide solo el ángulo entre vectores, lo que lo hace ideal para embeddings de texto, y particularmente para modelos de embeddings como los de [OpenAI cuyos valores vienen normalizados](https://help.openai.com/en/articles/8984345-which-distance-function-should-i-use).

Como es fácil ver en su expresión, esta métrica de distancia devuelve valores en el rango $[0, 2]$. Cuanto más cerca de 0, más similares serán los vectores $A$ y $B$. Esto nos deja en la necesidad de definir una métrica de `relevance_score` personalizada, que nos permita reescalar los resultados en valores comprendidos entre $[0, 1]$, siendo los valores más cercanos a 1 indicativo de mayor similitud entre los vectores. Esta métrica la hemos definido en la función `cosine_distance_relevance_score_fn` de la siguiente manera:

$$R_{score} = 1 - \frac{Distancia}{2}$$



In [8]:
chroma_openai_ephemeral_client = chromadb.EphemeralClient()
openai_embeddings = AzureOpenAIEmbeddings(model=AZURE_OPENAI_EMBEDDING_MODEL)


with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)


tables_summary_chunks = chunk_mdl_by_table_summary(mdl_data)
if tables_summary_chunks:
    try:
        chroma_openai_ephemeral_client.delete_collection(TABLES_SUMMARY_COLLECTION_NAME)
    except:
        pass

    tables_summary_collection = Chroma(
        client= chroma_openai_ephemeral_client,
        collection_name= TABLES_SUMMARY_COLLECTION_NAME,
        embedding_function= openai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(tables_summary_chunks, tables_summary_collection)


columns_chunks = chunk_mdl_by_column(mdl_data)
if columns_chunks:
    try:
        chroma_openai_ephemeral_client.delete_collection(COLUMNS_COLLECTION_NAME)
    except:
        pass

    columns_collection = Chroma(
        client= chroma_openai_ephemeral_client,
        collection_name= COLUMNS_COLLECTION_NAME,
        embedding_function= openai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(columns_chunks, columns_collection)


##### Hierarchical RAG

Dada la naturaleza de nuestras colecciones, donde las columnas existen dentro del contexto de cada tabla, nos proponemos a utilizar la técnica de [**RAG jerárquico**](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/hierarchical_indices.ipynb) sobre las coleciones creadas. Esto significa que, primero buscaremos las tablas que sean más relevantes semánticamente para la query del usuario (en la coleción `tables_summary`) y, luego, para cada tabla obtenida, buscaremos sus columnas más relevantes (en la coleción `collumns`, filtrando la metada para que coincida con el dato de la tabla):

In [9]:

openai_embeddings = AzureOpenAIEmbeddings(model=AZURE_OPENAI_EMBEDDING_MODEL)

tables_summary_collection = Chroma(
    client= chroma_openai_ephemeral_client,
    collection_name= TABLES_SUMMARY_COLLECTION_NAME,
    embedding_function= openai_embeddings,
    relevance_score_fn= cosine_distance_relevance_score_fn,
    create_collection_if_not_exists= False
)

columns_collection = Chroma(
    client= chroma_openai_ephemeral_client,
    collection_name= COLUMNS_COLLECTION_NAME,
    embedding_function= openai_embeddings,
    relevance_score_fn= cosine_distance_relevance_score_fn,
    create_collection_if_not_exists= False
)

In [10]:
QUERY = 'Los 10 artículos más comprados.'

In [11]:
tables_summary_results = tables_summary_collection.similarity_search_with_relevance_scores(
    query= QUERY,
    k= 10,
    score_threshold= 0.20
)

relevant_columns = []
for table_result in tables_summary_results:
    database_name = table_result[0].metadata['database_name']
    schema_name = table_result[0].metadata['schema_name']
    table_name = table_result[0].metadata['table_name']

    table_filter = {
        "$and": [
            {"database_name": {"$eq": database_name}},
            {"schema_name": {"$eq": schema_name}},
            {"table_name": {"$eq": table_name}}
        ]
    }

    columns_results = columns_collection.similarity_search_with_relevance_scores(
        query= QUERY,
        k= 15,
        filter=table_filter,
        score_threshold= 0.15
    )

    if columns_results:
        relevant_columns.append({
            'table_summary': {
                'content': table_result[0].page_content,
                'relevance_score': table_result[1]
            },
            'columns': [{'content': col[0].page_content, 'relevance_score': col[1]} for col in columns_results]
        })

if not relevant_columns:
    relevant_columns = [f'No docs retrieved for the query "{QUERY}"']

print(json.dumps(relevant_columns, indent=2, ensure_ascii=False))

No relevant docs were retrieved using the relevance score threshold 0.15


[
  {
    "table_summary": {
      "content": "Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promot

In [12]:
print(relevant_columns)

[{'table_summary': {'content': 'Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promotion_key)\n- (sa

In [13]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0)

SYSTEM_PROMPT = """
You are a context summarizer that receives retrieval documents in JSON format.
Your role is to:

1. Analyze the given JSON documents carefully.
2. Identify and extract only the information relevant to answer the user's question.
3. Summarize relevant tables, columns, keys, and descriptions into a clear and concise text.
4. Exclude any irrelevant or low-relevance details.
5. Ensure factual accuracy; do not fabricate information.
6. Do NOT generate or suggest any SQL queries.
7. When it is possible, match the language of the query fields to the user's query. For example, if user's query is in spanish, use `spanish_...`.
8. Match the language of the summary to the user's query.

Input:
- Retrieval documents (JSON): {retrieval_documents}

Output:
- A concise summary of relevant context—focused on tables, columns, keys, and descriptions—that will help generate an accurate SQL query. Don't suggest any query, only bring information in plain text. 
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieval_documents': json.dumps(relevant_columns, ensure_ascii=False)
})

Markdown(contexto_resumido)

Para obtener información sobre los 10 artículos más comprados, se pueden considerar las siguientes tablas y columnas relevantes de la base de datos `adventure_works_dw` en el esquema `sales`:

1. **Tabla: fact_sales**
   - **Descripción**: Tabla de hechos que contiene el detalle de las órdenes de ventas entregadas, con granularidad a nivel de línea.
   - **Claves Primarias**: 
     - `sales_order_number`
     - `sales_order_line_number`
   - **Columnas relevantes**:
     - `order_quantity`: Cantidad vendida de cada producto.
     - `product_key`: Clave del producto (relacionada con la tabla `dim_product`).

2. **Tabla: dim_product**
   - **Descripción**: Tabla de dimensión que almacena información detallada de cada producto vendido.
   - **Clave Primaria**: 
     - `product_key`
   - **Columnas relevantes**:
     - `english_product_name`: Nombre del producto en inglés.
     - `spanish_product_category_name`: Categoría principal a la que pertenece el producto en español.

Para identificar los artículos más comprados, se debe analizar la columna `order_quantity` de la tabla `fact_sales` y relacionarla con la tabla `dim_product` para obtener los nombres de los productos.

#### LangChain: ChromaDB + Google GenAI Embeddings

Ahora haremos una segunda implementación de prueba de Chroma valiéndonos de funcionalidades de la librería LangChain, usando un modelo de embeddings de Google Gemini a través de Gemini API, pero trabajando con VertexAI de fondo, lo que nos permite asegurar que los datos permanezcan en la region / location deseada.

Dado que la clase implementada por LangChain para utilizar embeddings de Gemini API no soporta usar VertexAI por detrás, hemos procedido a implementar una clase personalizada que sí nos los permite, a la que hemos llamado `GenAIExtendedEmbeddingFunction`.

Generamos una instancia de la clase y verificamos que los parámetros son los esperados:

In [14]:
genai_embeddings = GenAIExtendedEmbeddingFunction(model= GENAI_EMBEDDING_MODEL)
print(f'VertexAI: {genai_embeddings.genai_client._api_client.vertexai}')
print(f'Project: {genai_embeddings.genai_client._api_client.project}')
print(f'Location: {genai_embeddings.genai_client._api_client.location}')
print(f'Auth Tokens: {genai_embeddings.genai_client.auth_tokens}')

VertexAI: True
Project: gen-lang-client-0151767776
Location: europe-west1
Auth Tokens: <google.genai.tokens.Tokens object at 0x000001776D59C810>


El próximo paso es crear un cliente persistente de ChromaDB, e inicializar las colecciones tanto a nivel de tablas como de columnas. Al inicializar las colecciones, especificaremos que queremos usar como métrica de distancia la distancia de Coseno, que se calcula de la siguiente manera:

$$Distancia = 1 - \frac{A \cdot B}{\|A\| \|B\|}$$

Donde:
* $A \cdot B$ es el producto punto (producto escalar) de los vectores $A$ y $B$.
* $∥A∥$ y $∥B∥$ son las magnitudes (o normas euclidianas) de los vectores $A$ y $B$, respectivamente.
​

La métrica **Distancia del Coseno** mide solo el ángulo entre vectores, lo que lo hace ideal para embeddings de texto, y particularmente para modelos de embeddings como el que usaremos [(`gemini-embedding-001`) cuyos valores vienen normalizados](https://ai.google.dev/gemini-api/docs/embeddings#quality-for-smaller-dimensions).

Como es fácil ver en su expresión, esta métrica de distancia devuelve valores en el rango $[0, 2]$. Cuanto más cerca de 0, más similares serán los vectores $A$ y $B$. Esto nos deja en la necesidad de definir una métrica de `relevance_score` personalizada, que nos permita reescalar los resultados en valores comprendidos entre $[0, 1]$, siendo los valores más cercanos a 1 indicativo de mayor similitud entre los vectores. Esta métrica la hemos definido en la función `cosine_distance_relevance_score_fn` de la siguiente manera:

$$R_{score} = 1 - \frac{Distancia}{2}$$

In [15]:
chroma_genai_ephemeral_client = chromadb.EphemeralClient()
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)


with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)


tables_summary_chunks = chunk_mdl_by_table_summary(mdl_data)
if tables_summary_chunks:
    try:
        chroma_genai_ephemeral_client.delete_collection(TABLES_SUMMARY_COLLECTION_NAME)
    except:
        pass

    tables_summary_collection = Chroma(
        client= chroma_genai_ephemeral_client,
        collection_name= TABLES_SUMMARY_COLLECTION_NAME,
        embedding_function= genai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(tables_summary_chunks, tables_summary_collection)


columns_chunks = chunk_mdl_by_column(mdl_data)
if columns_chunks:
    try:
        chroma_genai_ephemeral_client.delete_collection(COLUMNS_COLLECTION_NAME)
    except:
        pass

    columns_collection = Chroma(
        client= chroma_genai_ephemeral_client,
        collection_name= COLUMNS_COLLECTION_NAME,
        embedding_function= genai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(columns_chunks, columns_collection)


##### Hierarchical RAG

Dada la naturaleza de nuestras colecciones, donde las columnas existen dentro del contexto de cada tabla, nos proponemos a utilizar la técnica de [**RAG jerárquico**](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/hierarchical_indices.ipynb) sobre las coleciones creadas. Esto significa que, primero buscaremos las tablas que sean más relevantes semánticamente para la query del usuario (en la coleción `tables_summary`) y, luego, para cada tabla obtenida, buscaremos sus columnas más relevantes (en la coleción `collumns`, filtrando la metada para que coincida con el dato de la tabla):

In [16]:
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)

tables_summary_collection = Chroma(
    client= chroma_genai_ephemeral_client,
    collection_name= TABLES_SUMMARY_COLLECTION_NAME,
    relevance_score_fn= cosine_distance_relevance_score_fn,
    embedding_function= genai_embeddings
)

columns_collection = Chroma(
    client= chroma_genai_ephemeral_client,
    collection_name= COLUMNS_COLLECTION_NAME,
    relevance_score_fn= cosine_distance_relevance_score_fn,
    embedding_function= genai_embeddings
)

In [17]:
QUERY = 'Los 10 artículos más comprados.'

In [18]:
tables_summary_results = tables_summary_collection.similarity_search_with_relevance_scores(
    query= QUERY,
    k= 10,
    score_threshold= 0.75
)

relevant_columns = []
for table_result in tables_summary_results:
    database_name = table_result[0].metadata['database_name']
    schema_name = table_result[0].metadata['schema_name']
    table_name = table_result[0].metadata['table_name']

    table_filter = {
        "$and": [
            {"database_name": {"$eq": database_name}},
            {"schema_name": {"$eq": schema_name}},
            {"table_name": {"$eq": table_name}}
        ]
    }

    columns_results = columns_collection.similarity_search_with_relevance_scores(
        query= QUERY,
        k= 15,
        filter=table_filter,
        score_threshold= 0.75
    )

    if columns_results:
        relevant_columns.append({
            'table_summary': {
                'content': table_result[0].page_content,
                'relevance_score': table_result[1]
            },
            'columns': [{'content': col[0].page_content, 'relevance_score': col[1]} for col in columns_results]
        })

if not relevant_columns:
    relevant_columns = [f'No docs retrieved for the query "{QUERY}"']

print(json.dumps(relevant_columns, indent=2, ensure_ascii=False))

No relevant docs were retrieved using the relevance score threshold 0.75


[
  {
    "table_summary": {
      "content": "Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promot

In [19]:
print(relevant_columns)

[{'table_summary': {'content': 'Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promotion_key)\n- (sa

In [20]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0)

SYSTEM_PROMPT = """
You are a context summarizer that receives retrieval documents in JSON format.
Your role is to:

1. Analyze the given JSON documents carefully.
2. Identify and extract only the information relevant to answer the user's question.
3. Summarize relevant tables, columns, keys, and descriptions into a clear and concise text.
4. Exclude any irrelevant or low-relevance details.
5. Ensure factual accuracy; do not fabricate information.
6. Do NOT generate or suggest any SQL queries.
7. When it is possible, match the language of the query fields to the user's query. For example, if user's query is in spanish, use `spanish_...`.
8. Match the language of the summary to the user's query.

Input:
- Retrieval documents (JSON): {retrieval_documents}

Output:
- A concise summary of relevant context—focused on tables, columns, keys, and descriptions—that will help generate an accurate SQL query. Don't suggest any query, only bring information in plain text. 
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieval_documents': json.dumps(relevant_columns, ensure_ascii=False)
})

Markdown(contexto_resumido)

Para obtener información sobre los 10 artículos más comprados, se pueden considerar las siguientes tablas y columnas relevantes de la base de datos `adventure_works_dw` en el esquema `sales`:

1. **Tabla: fact_sales**
   - **Descripción**: Contiene el detalle de las órdenes de ventas entregadas, con granularidad a nivel de línea.
   - **Claves Primarias**: 
     - sales_order_number
     - sales_order_line_number
   - **Columnas relevantes**:
     - **order_quantity**: Cantidad vendida de cada producto.
     - **product_key**: Clave del producto (relacionada con la tabla `dim_product`).

2. **Tabla: dim_product**
   - **Descripción**: Almacena información detallada de cada producto vendido.
   - **Clave Primaria**: 
     - product_key
   - **Columnas relevantes**:
     - **english_product_name**: Nombre del producto (en inglés).
     - **spanish_product_name**: Nombre del producto (en español).
     - **list_price**: Precio de venta del producto en USD.

Para identificar los artículos más comprados, se puede sumar la columna `order_quantity` de la tabla `fact_sales` agrupada por `product_key` y luego unirse a la tabla `dim_product` para obtener los nombres de los productos.

#### Conclusiones e Implementación Final

Como podemos ver, siguiendo el mismo proceso para ambos modelos de embeddings, el modelo `gemini-embedding-001` de Google, aunque parece devolver resultados similares que los que se obtienen con `text-embedding-3-small` de OpenAI, consigue mejores resultados en términos de relevancia. Esto nos hace pensar que es mejor al capturar la semántica de nuestro documentos frente a la de las consultas.

Esto puede deberse, en gran medida, a que , aunque ambos modelos presentan un **tamaño de embeddings de 3072**, el modelo de Google presenta un rendimiento destacable en tareas multilingües, como puede apreciarse en el [paper de su presentación](https://arxiv.org/html/2503.07891v1). Este es un punto no menor al trabajar nuestra documentación, ya que la misma mezcla conceptos en inglés (nombres de tablas y campos) y en español (descripciones), como asi también al momento de consultar, que podrá hacerse en cualquiera de estos 2 idiomas de manera indistinta.

Teniendo en cuenta estas conclusiones, finalmente, **procedemos a implementar nuestras colecciones de `ChromaDB` de manera definitiva usando el modelo `gemini-embedding-001`**.

In [22]:
chroma_client = chromadb.PersistentClient(path= CHROMA_DB_PATH)
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)


with open(MDL_PATH, 'r', encoding='utf-8') as mdl_file:
    mdl_data = yaml.safe_load(mdl_file)


tables_summary_chunks = chunk_mdl_by_table_summary(mdl_data)
if tables_summary_chunks:
    try:
        chroma_client.delete_collection(TABLES_SUMMARY_COLLECTION_NAME)
    except:
        pass

    tables_summary_collection = Chroma(
        client= chroma_client,
        collection_name= TABLES_SUMMARY_COLLECTION_NAME,
        embedding_function= genai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(tables_summary_chunks, tables_summary_collection)


columns_chunks = chunk_mdl_by_column(mdl_data)
if columns_chunks:
    try:
        chroma_client.delete_collection(COLUMNS_COLLECTION_NAME)
    except:
        pass

    columns_collection = Chroma(
        client= chroma_client,
        collection_name= COLUMNS_COLLECTION_NAME,
        embedding_function= genai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(columns_chunks, columns_collection)


---

## Fichero MarkDown con Reglas de Negocio

El objetivo de esta sección es procesar un fichero MarkDown que contien lógica de negocio. Para esto, debemos cargarlo, generar los chunks de una manera que sean semánticamente relevantes, generar su metadata si creemos que corresponde, y, finalmente generar sus embeddings y almacenarlos en una colescción de nuestra base de datos vectorial ChromaDB.

Como primer paso, procedemos a cargar el fichero y a ver un poco de su aspecto:

In [29]:
md_loader = TextLoader(BUSINESS_RULES_PATH, encoding='utf-8')
md_text = md_loader.load()[0].page_content

Markdown(md_text[500:2000])

 y la operacionalización de la inteligencia de negocio.

## 1. Representación de Entidades y Preferencias Lingüísticas

Cuando se solicita conceptualmente una entidad, como "productos", "clientes", "promociones" o "revendedores", se establece como principio operativo priorizar los campos que finalizan en `_name` para su adecuada representación en las salidas de las consultas SQL. Adicionalmente, se procurará mantener la coherencia con el idioma en que la consulta original fue formulada.

- **Principio Operativo:** Para la visualización de entidades en los resultados de consultas, se debe seleccionar el campo `_name` que mejor represente la entidad. En el caso de campos bilingües donde la versión en español (`spanish_..._name`, `spanish_..._description`) pueda ser nula y la consulta se realice en español, se utilizará la versión en inglés como alternativa (`fallback`) para asegurar la disponibilidad del dato y mantener la representación de la entidad.
- **Lógica SQL (Ejemplo para `dim_product`):**

    ```sql
    -- Selección del nombre del producto en español o, si es NULL, en inglés, para su representación
    SELECT
        product_key,
        COALESCE(spanish_product_name, english_product_name) AS product_name_localized,
        COALESCE(spanish_product_subcategory_name, english_product_subcategory_name) AS product_subcategory_name_localized,
        COALESCE(spanish_product_category_name, english_product_category_name) AS product_category_name_localized
    FROM
        

### Chunking

Ahora definimos algunas funciones que nos permitirán actualizar el contenido MarkDown de nuestro fichero, y de sus chunks. Esto es necesario debido a que la clase nativa de LangChain `MarkdownHeaderTextSplitter` no solo que no "limpia" el contenido del MarkDown, si no que también internamente durante su procesamiento utiliza `.strip()` sobre las líneas que conforman el texto MarkDown, lo que hace que se pierda la jerarquía del mismo.

In [30]:
def get_md_headers_tuples(md_doc: str, init_level: int = 2) -> list[tuple[str]]:
    """Generates a list of Markdown header tuples based on their presence in the text.

    This function iterates through Markdown header levels (starting from `init_level`)
    and creates a tuple for each level found in the text. The tuple contains
    the header syntax (e.g., '#', '##') and a corresponding descriptive name.
    It's useful for generating a configuration list for Markdown splitters,
    allowing them to correctly identify and process different header levels.

    Args:
        md_text (str): The input Markdown string to be checked for headers.
        init_level (int, optional): The starting header level to search for. 
                                    Defaults to 2 (for '##').

    Returns:
        list[tuple[str]]: A list of tuples, where each tuple contains the header syntax
                          and its descriptive name (e.g., ('#', 'title')).
    """
    
    headers_tuples = []

    while md_doc.find('#' * init_level) != -1:
        header_name = f'{init_level}_'

        if init_level == 1:
            header_name += 'title'
        if init_level == 2:
            header_name += 'subtitle'
        elif init_level == 3:
            header_name += 'section'
        elif init_level == 4:
            header_name += 'sub-section'
        elif init_level == 5:
            header_name += 'detail'
        else :
            header_name += f'minimum'

        headers_tuples.append(('#' * init_level, header_name))
        init_level += 1
    
    return headers_tuples



def add_trailing_dot_to_md(md_text: str) -> str:
    """Adds a trailing dot to each line of a Markdown string.

    This function adds a dot at the beginning of each line, except for
    Markdown headers (lines starting with one or more '#' followed by a space).
    This can be useful for text processing that requires unique line identifiers.

    Args:
        md_text (str): The input Markdown string.

    Returns:
        str: The Markdown string with a dot added to the beginning of each line.
    """

    updated_lines = []

    for line in md_text.split('\n'):
        updated_line = line

        if not regex.match(r'^#+\s.+$', line):
            updated_line = '.' + line

        updated_lines.append(updated_line)

    return '\n'.join(updated_lines)



def remove_trailing_dot_to_md(md_doc: Document) -> Document:
    """Removes a leading dot from each line of a Markdown document.

    This function processes each line of a document and removes the first
    character if it is a dot. This is the inverse of `add_trailing_dot_to_md`.

    Args:
        md_doc (Document): The input Markdown document object.

    Returns:
        Document: A new Document object with the leading dots removed.
    """
    
    updated_lines = []

    for line in md_doc.page_content.split('\n'):
        updated_line = line

        if regex.match(r'^\..*$', line):
            updated_line = line[1:]

        updated_lines.append(updated_line)

    return Document(page_content='\n'.join(updated_lines), metadata= md_doc.metadata)



def clean_md_text_formatting(md_doc: Document) -> Document:
    """Removes common Markdown formatting from a document.

    This function processes a Markdown document line by line to remove
    various formatting syntax, including headers, bold, italics,
    strikethrough, and horizontal rules. It preserves the basic
    structure of lists and their prefixes.

    Args:
        md_doc (Document): The input Markdown document object with formatting.

    Returns:
        Document: A new Document object with the Markdown formatting removed.
    """
    
    cleaned_lines = []

    last_line_divisor = False

    for line in md_doc.page_content.split('\n'):
        if last_line_divisor:
            last_line_divisor = False
            continue

        if regex.match(r'^\s*(---|\*\*\*|\+\+\+)\s*$', line):
            last_line_divisor = True
            continue

        list_prefix = regex.match(r'^(\s*[-*+])\s', line)
        prefix = list_prefix.group(1) if list_prefix else ''
        content = line[len(prefix):]

        content = regex.sub(r'^\s*#+\s(.+)$', r'\1', content)
        content = regex.sub(r'(?<=^|\s|\p{P})\*\*\*(.*?)\*\*\*(?=\s|\p{P}|$)', r'\1', content)
        content = regex.sub(r'(?<=^|\s|\p{P})\*\*(.*?)\*\*(?=\s|\p{P}|$)', r'\1', content)
        content = regex.sub(r'(?<=^|\s|\p{P})\*(.*?)\*(?=\s|\p{P}|$)', r'\1', content)
        content = regex.sub(r'(?<=^|\s|\p{P})___(.*?)___(?=\s|\p{P}|$)', r'\1', content)
        content = regex.sub(r'(?<=^|\s|\p{P})__(.*?)__(?=\s|\p{P}|$)', r'\1', content)
        content = regex.sub(r'(?<=^|\s|\p{P})_(.*?)_(?=\s|\p{P}|$)', r'\1', content)
        content = regex.sub(r'(?<=^|\s|\p{P})~(.*?)~(?=\s|\p{P}|$)', r'\1', content)
        content = regex.sub(r'(?<=^|\s|\p{P})<u>(.*?)</u>(?=\s|\p{P}|$)', r'\1', content)

        cleaned_lines.append(prefix + content)

    return Document(page_content='\n'.join(cleaned_lines), metadata= md_doc.metadata)



def add_all_headers_to_metadata(docs: list[Document], headers_tuples: list[tuple[str]]) -> list[Document]:
    """Adds all specified header keys to the metadata of each document.

    This function ensures that a deep copy of each document's metadata
    contains all header keys specified in `headers_tuples`. If a key is not
    present, it is added with a value of 'N/A'. This is useful for
    standardizing the metadata schema across all documents.

    Args:
        docs (list[Document]): A list of LangChain Document objects.
        headers_tuples (list[tuple[str, str]]): A list of tuples, where the
            second element is the key to be added to the document's metadata
            (e.g., [("Title", "title")]).

    Returns:
        list[Document]: A new list of Document objects with standardized metadata.
    """
    if not docs:
        return []

    final_documents = deepcopy(docs)

    for doc in final_documents:
        for _, header_key in headers_tuples:
            if header_key not in doc.metadata:
                doc.metadata[header_key] = 'N/A'

    return final_documents



def add_headers_comb_chunk_to_metadata(docs: list[Document], headers_tuples: list[tuple[str]]) -> list[Document]:
    """
    Adds a chunk ID and the total number of chunks for each unique
    combination of header values to the documents' metadata.

    This function performs two passes over the documents. The first pass
    counts the number of chunks for each unique header combination. The
    second pass then assigns a sequential ID to each chunk and adds the
    total count to the metadata.

    Args:
        docs (list[Document]): A list of LangChain Document objects.
        headers_tuples (list[tuple[str, str]]): A list of tuples, where the
            second element is the key to extract from the document's metadata
            (e.g., [("Title", "title")]).

    Returns:
        list[Document]: A new list of Document objects with the chunk ID
                        and total count added to their metadata.
    """
    
    if not docs:
        return []
    
    final_documents = deepcopy(docs)

    combination_counts = {}
    for doc in final_documents:
        current_headers_tuple = tuple(doc.metadata.get(header_key) for _, header_key in headers_tuples)
        combination_counts[current_headers_tuple] = combination_counts.get(current_headers_tuple, 0) + 1

    headers_comb_chunk_ids = {}
    for doc in final_documents:
        current_headers_tuple = tuple(doc.metadata.get(header_key) for _, header_key in headers_tuples)
        chunk_id = headers_comb_chunk_ids.get(current_headers_tuple, 0)

        doc.metadata[HEADERS_CHUNK_ID_KEY] = chunk_id
        doc.metadata[HEADERS_CHUNKS_TOTAL_KEY] = combination_counts[current_headers_tuple]

        headers_comb_chunk_ids[current_headers_tuple] = chunk_id + 1

    return final_documents



def add_chunk_overlap_to_metadata(docs: list[Document], chunk_overlap: int) -> list[Document]:
    """Adds a chunk overlap value to the metadata of each document.

    This function iterates through a list of LangChain Document objects and adds
    a specified chunk overlap value to the metadata dictionary of each document.
    The process creates a deep copy of the original documents to prevent
    modifying them in place.

    Args:
        docs (list[Document]): A list of LangChain Document objects.
        chunk_overlap (int): The integer value representing the chunk overlap.

    Returns:
        list[Document]: A new list of Document objects with the chunk overlap
                        value added to their metadata.
    """

    if not docs:
        return []
    
    
    final_documents = deepcopy(docs)

    for doc in final_documents:
        doc.metadata[CHUNK_OVERLAP_KEY] = chunk_overlap

    return final_documents

Utilizaremos como primer método de chunking un `MarkdownHeaderTextSplitter`, para luego aplicar un `RecursiveCharacterTextSplitter` que nos permita llevar los chunks al tamaño deseado. Finalmente, añadiremos en la metadata de cada chunk un `id` que lo identifique de forma única dentro de la combinación de headers que presente en la metadata.

In [31]:
def get_business_logic_chunks(md_text: str) -> list[Document]:
    """Splits a Markdown document into semantically meaningful chunks based on business logic.

    This function is a pipeline that processes a Markdown text by:
    1. Identifying header levels to create splitting points.
    2. Adding a temporary leading dot to all non-header lines to preserve them during splitting.
    3. Splitting the document by Markdown headers using `MarkdownHeaderTextSplitter`.
    4. Cleaning up formatting and removing the temporary leading dots.
    5. Further splitting the resulting chunks using `RecursiveCharacterTextSplitter` to ensure they do not exceed a predefined size.
    6. Adding combined headers and a chunk ID to the final documents' metadata.

    Args:
        md_text (str): The input Markdown text as a single string.

    Returns:
        list[Document]: A list of Document objects, each representing a
                        processed chunk of the original Markdown text.
    """
    
    headers_to_split_on = get_md_headers_tuples(md_text)

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers= False,
        return_each_line= False,
    )

    recursive_splitter = RecursiveCharacterTextSplitter(
        separators= Language.MARKDOWN,
        chunk_size= BUSINESS_RULES_CHUNK_SIZE,
        chunk_overlap= BUSINESS_RULES_CHUNK_OVERLAP
    )

    md_header_documents = markdown_splitter.split_text(add_trailing_dot_to_md(md_text))

    md_header_documents = [clean_md_text_formatting(remove_trailing_dot_to_md(md_doc))
                        for md_doc in md_header_documents if md_doc.metadata]

    final_documents = recursive_splitter.split_documents(md_header_documents)

    final_documents = add_all_headers_to_metadata(final_documents, headers_to_split_on)
    final_documents = add_headers_comb_chunk_to_metadata(final_documents, headers_to_split_on)
    final_documents = add_chunk_overlap_to_metadata(final_documents, BUSINESS_RULES_CHUNK_OVERLAP)

    return final_documents

In [32]:
get_business_logic_chunks(md_text= md_text)

[Document(metadata={'2_subtitle': '1. Representación de Entidades y Preferencias Lingüísticas', '3_section': 'N/A', 'headers_chunk_id': 0, 'headers_comb_total': 3, 'chunk_overlap': 150}, page_content='1. Representación de Entidades y Preferencias Lingüísticas\n\nCuando se solicita conceptualmente una entidad, como "productos", "clientes", "promociones" o "revendedores", se establece como principio operativo priorizar los campos que finalizan en `_name` para su adecuada representación en las salidas de las consultas SQL. Adicionalmente, se procurará mantener la coherencia con el idioma en que la consulta original fue formulada.\n\n- Principio Operativo: Para la visualización de entidades en los resultados de consultas, se debe seleccionar el campo `_name` que mejor represente la entidad. En el caso de campos bilingües donde la versión en español (`spanish_...na'),
 Document(metadata={'2_subtitle': '1. Representación de Entidades y Preferencias Lingüísticas', '3_section': 'N/A', 'headers

### LangChain: ChromaDB + Google GenAI Embeddings

Ahora sí, procedemos a generar los embeddings de cada chunk y a almacenarlos en nuestra base de datos vectorial de `ChromaDB`. Esta vez, siguiendo la línea de las conclusiones obtenidas en el apartado anterior, utilizaremos un modelo de embeddings de Google Gemini a través de Gemini API, pero trabajando con VertexAI de fondo, lo que nos permite asegurar que los datos permanezcan en la region / location deseada.

Dado que la clase implementada por LangChain para utilizar embeddings de Gemini API no soporta usar VertexAI por detrás, nuevamente, utilizaremos la clase `GenAIExtendedEmbeddingFunction` creada anteriormente.

Generamos una instancia de la clase y verificamos que los parámetros son los esperados:

In [33]:
genai_embeddings = GenAIExtendedEmbeddingFunction(model= GENAI_EMBEDDING_MODEL)
print(f'VertexAI: {genai_embeddings.genai_client._api_client.vertexai}')
print(f'Project: {genai_embeddings.genai_client._api_client.project}')
print(f'Location: {genai_embeddings.genai_client._api_client.location}')
print(f'Auth Tokens: {genai_embeddings.genai_client.auth_tokens}')

VertexAI: True
Project: gen-lang-client-0151767776
Location: europe-west1
Auth Tokens: <google.genai.tokens.Tokens object at 0x000002025AC0C150>


El próximo paso es crear un cliente persistente de ChromaDB, e inicializar la colección correspondiente con la lógica de negocio. Al inicializar la colección, nuevamente, especificaremos que queremos usar como métrica de distancia la distancia de Coseno.

In [34]:
chroma_client = chromadb.PersistentClient(path= CHROMA_DB_PATH)
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)


business_logic_chunks = get_business_logic_chunks(md_text)
if business_logic_chunks:
    try:
        chroma_client.delete_collection(BUSINESS_LOGIC_COLLECTION_NAME)
    except:
        pass

    business_logic_collection = Chroma(
        client= chroma_client,
        collection_name= BUSINESS_LOGIC_COLLECTION_NAME,
        embedding_function= genai_embeddings,
        collection_configuration= {'hnsw': {'space': 'cosine'}},
        create_collection_if_not_exists= True
    )

    add_docs_to_chroma_col(business_logic_chunks, business_logic_collection)


Ahora lanzaremos una consulta a la colección creada, para ver que los resultados obtenidos muestren cierto nivel de relevancia. Para esto, como ya hemos hecho en el apartado anterior, utilizaremoscomo métrica personalizada de `relevance_score` la función `cosine_distance_relevance_score_fn`.

In [9]:
chroma_client = chromadb.PersistentClient(path= CHROMA_DB_PATH)
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)

business_logic_collection = Chroma(
    client= chroma_client,
    collection_name= BUSINESS_LOGIC_COLLECTION_NAME,
    relevance_score_fn= cosine_distance_relevance_score_fn,
    embedding_function= genai_embeddings
)

In [10]:
QUERY = 'Los 10 artículos más comprados.'

In [11]:
business_logic_results = business_logic_collection.similarity_search_with_relevance_scores(
    query= QUERY,
    k= 5,
    score_threshold= 0.8
)

business_logic_results

[(Document(id='fc485630-d27a-49fb-926d-ebbce28bbe2b', metadata={'chunk_overlap': 150, 'headers_chunk_id': 0, 'headers_comb_total': 3, '2_subtitle': '3. Métricas Clásicas del Retail y su Interpretación', '3_section': 'N/A'}, page_content='3. Métricas Clásicas del Retail y su Interpretación\n\nEs crucial entender cómo se miden las diferentes métricas en el contexto de negocio.\n\n- Cuando se habla de productos "más vendidos", sin especificar más, generalmente nos referimos a la cantidad de unidades (`order_quantity`) vendidas. También puede ser relevante medir esto por el importe facturado (`sales_amount`) para identificar productos que, aunque se vendan en menor volumen, generan más ingresos. Pero, en este último caso se especificarán términos como "de mayor facturación".\n- Cuando se refiere a clientes, tiendas/distribuidores o vendedores que "más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicional'),
  0.8411799669265747),
 (Document(id='e2

In [12]:
business_logic_results[0][0]

Document(id='fc485630-d27a-49fb-926d-ebbce28bbe2b', metadata={'chunk_overlap': 150, 'headers_chunk_id': 0, 'headers_comb_total': 3, '2_subtitle': '3. Métricas Clásicas del Retail y su Interpretación', '3_section': 'N/A'}, page_content='3. Métricas Clásicas del Retail y su Interpretación\n\nEs crucial entender cómo se miden las diferentes métricas en el contexto de negocio.\n\n- Cuando se habla de productos "más vendidos", sin especificar más, generalmente nos referimos a la cantidad de unidades (`order_quantity`) vendidas. También puede ser relevante medir esto por el importe facturado (`sales_amount`) para identificar productos que, aunque se vendan en menor volumen, generan más ingresos. Pero, en este último caso se especificarán términos como "de mayor facturación".\n- Cuando se refiere a clientes, tiendas/distribuidores o vendedores que "más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicional')

In [13]:
business_logic_results[2][0]

Document(id='6df22d72-6596-42bc-80ec-4983c7e1bd1b', metadata={'2_subtitle': '3. Métricas Clásicas del Retail y su Interpretación', 'chunk_overlap': 150, 'headers_chunk_id': 1, '3_section': 'N/A', 'headers_comb_total': 3}, page_content='más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicionalmente, se puede considerar la frecuencia de compra (para clientes y distribuidores) o el número de pedidos gestionados (para vendedores) como métricas complementarias.\n- Para el análisis de rentabilidad, la métrica clave es el margen bruto (`gross margin`) o el porcentaje de margen bruto (`gross margin percentage`), lo cual permite comprender la contribución real de cada venta o producto a las ganancias, más allá del volumen o ingreso total. Este enfoque es vital para evaluar la eficiencia operativa.\n- En el contexto de la gestión de promociones, el éxito no solo se mide por el discount_amount o average_discount_rate, sino ta')

#### Maximal Marginal Relevance (MMR)

Como podemos ver, los resultados obtenidos muestran valores altos de relevancia. Dado que para este caso, a diferencia del anterior, estamos trabajando con chunks que presentan overlapping, corremos el riesgo de que nuestros resultados sean redundantes, como se muestra en las 2 celdas anteriores. Ante esta situación, parece una buena idea utilizar `max_marginal_relevance_search`. 

Esta búsqueda aplica la técnica de [***Maximal Marginal Relevance (MMR)***](https://aclanthology.org/X98-1025/), mediante la que se intenta reducir la redundancia en los resultados mientras, al mismo tiempo, se mantiene la relevancia de los mismos. Es decir, un documento presentará una alta Relevancia Marginal si es a la vez relevante para la query y presenta una similitud reducida con los documentos previamente seleccionados.

In [14]:
business_logic_mmr_results = business_logic_collection.max_marginal_relevance_search(
    query= QUERY,
    k= 5,
    fetch_k= 20,
    lambda_mult= 0.5
)

business_logic_mmr_results

[Document(id='fc485630-d27a-49fb-926d-ebbce28bbe2b', metadata={'2_subtitle': '3. Métricas Clásicas del Retail y su Interpretación', 'headers_chunk_id': 0, 'chunk_overlap': 150, 'headers_comb_total': 3, '3_section': 'N/A'}, page_content='3. Métricas Clásicas del Retail y su Interpretación\n\nEs crucial entender cómo se miden las diferentes métricas en el contexto de negocio.\n\n- Cuando se habla de productos "más vendidos", sin especificar más, generalmente nos referimos a la cantidad de unidades (`order_quantity`) vendidas. También puede ser relevante medir esto por el importe facturado (`sales_amount`) para identificar productos que, aunque se vendan en menor volumen, generan más ingresos. Pero, en este último caso se especificarán términos como "de mayor facturación".\n- Cuando se refiere a clientes, tiendas/distribuidores o vendedores que "más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicional'),
 Document(id='e26ae92e-e276-4b31-963e-031

#### Context Enrichment Window

Como podemos observar en el resultado anterior, Parece ser que los 5 chunks que hemos obtenido, a diferencia de lo que ocurría antes, ninguno pertenece a la misma jerarquía (mismo `2_subtitle` y `3_section`), por lo que parece estar funcionando mejor. Utilizaremos entonces esta técnica, y la combinaremos con una técnica de [**RAG de contexto eriquecido**](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/context_enrichment_window_around_chunk.ipynb), lo que nos permitirá ampliar el contexto que circunda a los chunks obtenidos.

Para lograr esto, hemos implementado una clase personalizada que contine toda la lógica necesaria para generar un contexto enriquecido. Está definida en la sección `Clases personalizadas`, bajo el nombre `DocumentContextEnricher`.

In [15]:
enricher = DocumentContextEnricher(chroma_collection= business_logic_collection)

enriched_contexts = enricher.get_enriched_context_chunks(
    business_logic_mmr_results, 
    k_chunks_context_window= 3
)

print(enriched_contexts[2])

mparativa 'Año Actual vs. Año Pasado' (requiere CTEs para agregaciones)
    WITH CurrentYearSales AS (
        SELECT
            EXTRACT(MONTH FROM order_date) AS sales_month,
            SUM(sales_amount) AS current_year_sales
        FROM sales.fact_sales
        WHERE EXTRACT(YEAR FROM order_date) = EXTRACT(YEAR FROM CURRENT_DATE)
        GROUP BY 1
    ),
    LastYearSales AS (
        SELECT
            EXTRACT(MONTH FROM order_date) AS sales_month,
            SUM(sales_amount) AS last_year_sales
        FROM sales.fact_sales
        WHERE EXTRACT(YEAR FROM order_date) = EXTRACT(YEAR FROM CURRENT_DATE) - 1
        GROUP BY 1
    )
    SELECT
        COALESCE(CY.sales_month, LY.sales_month) AS sales_month,
        COALESCE(CY.current_year_sales, 0) AS current_year_sales,
        COALESCE(LY.last_year_sales, 0) AS last_year_sales,
        (COALESCE(CY.current_year_sales, 0) - COALESCE(LY.last_year_sales, 0)) AS sales_difference,
        (COALESCE(CY.current_year_sales, 0) - COALES

In [16]:
print(enriched_contexts)

['3. Métricas Clásicas del Retail y su Interpretación\n\nEs crucial entender cómo se miden las diferentes métricas en el contexto de negocio.\n\n- Cuando se habla de productos "más vendidos", sin especificar más, generalmente nos referimos a la cantidad de unidades (`order_quantity`) vendidas. También puede ser relevante medir esto por el importe facturado (`sales_amount`) para identificar productos que, aunque se vendan en menor volumen, generan más ingresos. Pero, en este último caso se especificarán términos como "de mayor facturación".\n- Cuando se refiere a clientes, tiendas/distribuidores o vendedores que "más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicionalmente, se puede considerar la frecuencia de compra (para clientes y distribuidores) o el número de pedidos gestionados (para vendedores) como métricas complementarias.\n- Para el análisis de rentabilidad, la métrica clave es el margen bruto (`gross margin`) o el porcentaje de mar

In [18]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0.25)

SYSTEM_PROMPT = """
You are a context summarizer that receives retrieval documents.
Your role is to:

1. Analyze the given retrieval documents, which contain business logic information.
2. Identify and extract only the business logic rules and definitions relevant to the user's question. This includes metrics, calculations, entity definitions, and specific operational principles.
3. Summarize these business logics into a clear, concise, and easy-to-understand text.
4. Exclude any irrelevant details like table or column names, keys, or direct SQL query examples.
5. Ensure factual accuracy; do not fabricate information.
6. The summary should be in plain text, exclusively focused on explaining the business rules. Do not mention any technical or database-related details.
7. Match the language of the summary to the user's query.

Input:
- Retrieval documents: {retrieval_documents}

Output:
- A concise summary of the business logic relevant to the user's query. The summary should be focused on explaining the business rules and definitions in a non-technical manner.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieval_documents': enriched_contexts
})

Markdown(contexto_resumido)

Para identificar los 10 artículos más comprados, se considera la cantidad de unidades vendidas como la métrica principal. Sin embargo, también es relevante analizar el importe facturado, ya que algunos productos pueden generar más ingresos a pesar de tener un menor volumen de ventas. En este caso, se utilizaría el término "artículos de mayor facturación" para referirse a esos productos. 

En resumen, la evaluación de los artículos más comprados se basa en la cantidad de unidades vendidas y, complementariamente, en el importe total de ventas generado por cada artículo.

---

## Fichero JSON con Ejemplos de Consultas